(sec-ray-data-preprocessor)=
# Preprocessor

{numref}`sec-ray-data-transform` 介绍了通用接口 `map()` 和 `map_batches()`。对于结构化的表格类数据，Ray Data 在 `map()` 和 `map_batches()` 基础上，增加了一个高阶的 API：预处理器（Preprocessor）。[Preprocessor](https://docs.ray.io/en/latest/data/api/preprocessor.html) 是一系列特征处理操作，可与机器学习模型训练和推理更好地结合。其使用方式与 scikit-learn 的 [sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) 非常相似，熟悉 scikit-learn 的用户可以快速迁移过来。对于非结构化数据，比如图片、视频等，仍然建议使用 `map()` 或者 `map_batches()`。